In [154]:
using LinearAlgebra
using TensorKit
using TensorKitSectors
using SymGroupRepresentations
const SGR = SymGroupRepresentations;

In [155]:
q = 5
S = SNIrrep{q};
irrep_gen = SGR.irreps_gen.S5;

In [156]:
# usual Potts tensor
function classical_potts(q::Int, β::Float64)
    V = ℂ^q
    A_potts = zeros(Float64, V ⊗ V ← V ⊗ V)
    for (i, j, k, l) in Iterators.product(fill(1:q, 4)...)
        E = -(Int(i == j) + Int(j == l) + Int(l == k) + Int(k == i))
        A_potts[i, j, k, l] = exp(-β * E)
    end
    return A_potts
end


classical_potts (generic function with 1 method)

In [157]:
T0 = classical_potts(q, 0.5);

In [158]:
# Permutation representation for generators 
# x1 = (1,2,...,n), x2 = (1,2)
function permrep_gen(q::Int)
    x1 = zeros(Float64, q, q)
    x2 = zeros(Float64, q, q)
    for i in 1:q
        x1[mod1(i + 1, q), i] = 1
    end
    x2[1, 2] = x2[2, 1] = 1
    for i in 3:q
        x2[i, i] = 1
    end
    return [x1, x2]
end

permrep_gen (generic function with 1 method)

In [159]:
permrep = permrep_gen(q);
for rep in permrep
    display(rep)
    ρ = TensorMap(rep, ℂ^q ← ℂ^q)
    @assert (ρ ⊗ ρ) * T0 * (ρ' ⊗ ρ') ≈ T0
end

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0

5×5 Matrix{Float64}:
 0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [160]:
f = hcat(map(1:2) do c
    SGR.get_intertwiners(irrep_gen[c], permrep)[1]
end...)
f = TensorMap(f, ℂ^q ← ℂ^q)
T = (f' ⊗ f') * T0 * (f ⊗ f);
T_arr = convert(Array, T);

In [161]:
for (rep1, rep2) in zip(irrep_gen[1], irrep_gen[2])
    rep = Matrix(SGR.block_diag(rep1, rep2))
    ρ = TensorMap(rep, ℂ^q ← ℂ^q)
    @assert (ρ ⊗ ρ) * T * (ρ' ⊗ ρ') ≈ T
end

In [162]:
function get_reduced_elements_potts(T_arr)
    cs = [S([5]), S([4, 1])]
    slices = [1:1, 2:5]
    rdes = Vector{Pair{NTuple{q, S}, Float64}}()
    for (i1, i2, i3, i4) in Iterators.product(fill(1:2, 4)...)
        s1, s2, s3, s4 = cs[i1], cs[i2], cs[i3], cs[i4]
        for s in intersect(s1 ⊗ s2, s3 ⊗ s4)
            fin = fusiontensor(s3, s4, s)[:, :, :, 1]
            fout = fusiontensor(s1, s2, s)[:, :, :, 1]
            subT_arr = @view T_arr[slices[i1], slices[i2], slices[i3], slices[i4]]
            @tensor reduced_matrix[-1; -2] := conj(fout[1 2; -1]) * subT_arr[1 2; 3 4] * fin[3 4; -2]
            rde = reduced_matrix[1, 1]
            @assert isapprox(reduced_matrix, rde * I; atol = 1.0e-10)
            push!(rdes, (s1, s2, s3, s4, s) => rde)
        end
    end
    return Dict(rdes)
end


get_reduced_elements_potts (generic function with 1 method)

In [163]:
rdes = get_reduced_elements_potts(T_arr)

Dict{NTuple{5, S5Irrep}, Float64} with 15 entries:
  (4₁1₁, 5₁, 4₁1₁, 4₁1₁, 4₁1₁)   => 1.16856
  (5₁, 5₁, 5₁, 5₁, 5₁)           => 40.7533
  (4₁1₁, 5₁, 5₁, 4₁1₁, 4₁1₁)     => 1.09551
  (5₁, 4₁1₁, 4₁1₁, 5₁, 4₁1₁)     => 1.09551
  (4₁1₁, 4₁1₁, 4₁1₁, 4₁1₁, 4₁1₁) => 1.18232
  (4₁1₁, 4₁1₁, 4₁1₁, 4₁1₁, 3₁1₂) => 0.420839
  (4₁1₁, 4₁1₁, 4₁1₁, 4₁1₁, 5₁)   => 3.1195
  (5₁, 4₁1₁, 4₁1₁, 4₁1₁, 4₁1₁)   => 1.16856
  (4₁1₁, 4₁1₁, 5₁, 5₁, 5₁)       => 9.5199
  (5₁, 4₁1₁, 5₁, 4₁1₁, 4₁1₁)     => 4.75995
  (4₁1₁, 4₁1₁, 5₁, 4₁1₁, 4₁1₁)   => 1.16856
  (4₁1₁, 4₁1₁, 4₁1₁, 4₁1₁, 3₁2₁) => 0.420839
  (5₁, 5₁, 4₁1₁, 4₁1₁, 5₁)       => 9.5199
  (4₁1₁, 5₁, 4₁1₁, 5₁, 4₁1₁)     => 4.75995
  (4₁1₁, 4₁1₁, 4₁1₁, 5₁, 4₁1₁)   => 1.16856

In [167]:
Vsym = Vect[S](S([5]) => 1, S([4, 1]) => 1)
Tsym = zeros(Float64, Vsym ⊗ Vsym ← Vsym ⊗ Vsym);
for (f1, f2) in fusiontrees(Tsym)
    (s1, s2) = f1.uncoupled
    (s3, s4) = f2.uncoupled
    s = f1.coupled
    Tsym[f1, f2] .= rdes[(s1, s2, s3, s4, s)]
end

In [168]:
convert(Array, Tsym) ≈ T_arr

true